In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 23.9 MB/s eta 0:00:00


# Service 1

In [4]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import optuna
from optuna.samplers import TPESampler

# Load dataset
df = pd.read_csv("Service1.csv")

# Input and output
X = df[['latency_ms', 'cpu_usage_pct', 'memory_usage_pct', 'memory_allocated', 'cpu_allocated']]
y = df[['cpu_usage_pct', 'memory_usage_pct']]

# Split: 70% train, 15% val, 15% test
X_train_full, X_temp, y_train_full, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Scale features
x_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train_scaled = x_scaler.fit_transform(X_train_full)
X_val_scaled = x_scaler.transform(X_val)
X_test_scaled = x_scaler.transform(X_test)

y_train_scaled = y_scaler.fit_transform(y_train_full)
y_val_scaled = y_scaler.transform(y_val)

# Objective for Optuna
def objective(trial):
    C = trial.suggest_float('C', 0.1, 100.0, log=True)
    epsilon = trial.suggest_float('epsilon', 0.001, 1.0, log=True)
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

    svr_model = MultiOutputRegressor(SVR(C=C, epsilon=epsilon, gamma=gamma))
    svr_model.fit(X_train_scaled, y_train_scaled)
    preds_scaled = svr_model.predict(X_val_scaled)
    mse = mean_squared_error(y_val_scaled, preds_scaled)
    return mse

# Run optimization
study = optuna.create_study(direction="minimize", sampler=TPESampler())
study.optimize(objective, n_trials=30)

# Best model training
best_params = study.best_params
print("Best Parameters:", best_params)

final_svr = MultiOutputRegressor(SVR(**best_params))
final_svr.fit(np.vstack([X_train_scaled, X_val_scaled]), np.vstack([y_train_scaled, y_val_scaled]))

# Test set evaluation
y_test_scaled = y_scaler.transform(y_test)
preds_scaled = final_svr.predict(X_test_scaled)
preds = y_scaler.inverse_transform(preds_scaled)

mse = mean_squared_error(y_test, preds)
rmse = np.sqrt(mse)
print(f"\nTest MSE: {mse:.4f}")
print(f"Test RMSE: {rmse:.4f}")

# Sample prediction
sample = pd.DataFrame([[300, 0.25, 500, 40, 75]], columns=['latency_ms', 'cpu_usage_pct', 'memory_usage_pct', 'memory_allocated', 'cpu_allocated'])
sample_scaled = x_scaler.transform(sample)
predicted_scaled = final_svr.predict(sample_scaled)
predicted = y_scaler.inverse_transform(predicted_scaled)

print(f"\nSample Input: {sample.to_dict(orient='records')[0]}")
print(f"Predicted CPU, Memory Usage: CPU={predicted[0][0]:.3f}, Memory={predicted[0][1]:.2f}")


[I 2025-05-05 08:21:03,576] A new study created in memory with name: no-name-6fa11aa9-25d1-4ece-8c99-ee24812f9432
[I 2025-05-05 08:21:03,620] Trial 0 finished with value: 0.09990017461469881 and parameters: {'C': 0.16472847366485, 'epsilon': 0.3963709583595634, 'gamma': 'auto'}. Best is trial 0 with value: 0.09990017461469881.
[I 2025-05-05 08:21:03,642] Trial 1 finished with value: 0.7153657963371024 and parameters: {'C': 0.9423491122702939, 'epsilon': 0.9887151460943112, 'gamma': 'auto'}. Best is trial 0 with value: 0.09990017461469881.
[I 2025-05-05 08:21:03,804] Trial 2 finished with value: 0.0002857749470634865 and parameters: {'C': 1.5551943510006643, 'epsilon': 0.029562307741944592, 'gamma': 'auto'}. Best is trial 2 with value: 0.0002857749470634865.
[I 2025-05-05 08:21:04,386] Trial 3 finished with value: 2.2738308317972388e-05 and parameters: {'C': 2.8605720618531576, 'epsilon': 0.007861723220101535, 'gamma': 'auto'}. Best is trial 3 with value: 2.2738308317972388e-05.
[I 2025

Best Parameters: {'C': 43.89085922084776, 'epsilon': 0.001040573805193996, 'gamma': 'scale'}

Test MSE: 0.0004
Test RMSE: 0.0195

Sample Input: {'latency_ms': 300, 'cpu_usage_pct': 0.25, 'memory_usage_pct': 500, 'memory_allocated': 40, 'cpu_allocated': 75}
Predicted CPU, Memory Usage: CPU=63.786, Memory=29.43


# Service 2

In [5]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import optuna
from optuna.samplers import TPESampler

# Load dataset
df = pd.read_csv("Service2.csv")

# Input and output
X = df[['latency_ms', 'cpu_usage_pct', 'memory_usage_pct', 'memory_allocated', 'cpu_allocated']]
y = df[['cpu_usage_pct', 'memory_usage_pct']]

# Split: 70% train, 15% val, 15% test
X_train_full, X_temp, y_train_full, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Scale features
x_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train_scaled = x_scaler.fit_transform(X_train_full)
X_val_scaled = x_scaler.transform(X_val)
X_test_scaled = x_scaler.transform(X_test)

y_train_scaled = y_scaler.fit_transform(y_train_full)
y_val_scaled = y_scaler.transform(y_val)

# Objective for Optuna
def objective(trial):
    C = trial.suggest_float('C', 0.1, 100.0, log=True)
    epsilon = trial.suggest_float('epsilon', 0.001, 1.0, log=True)
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

    svr_model = MultiOutputRegressor(SVR(C=C, epsilon=epsilon, gamma=gamma))
    svr_model.fit(X_train_scaled, y_train_scaled)
    preds_scaled = svr_model.predict(X_val_scaled)
    mse = mean_squared_error(y_val_scaled, preds_scaled)
    return mse

# Run optimization
study = optuna.create_study(direction="minimize", sampler=TPESampler())
study.optimize(objective, n_trials=30)

# Best model training
best_params = study.best_params
print("Best Parameters:", best_params)

final_svr = MultiOutputRegressor(SVR(**best_params))
final_svr.fit(np.vstack([X_train_scaled, X_val_scaled]), np.vstack([y_train_scaled, y_val_scaled]))

# Test set evaluation
y_test_scaled = y_scaler.transform(y_test)
preds_scaled = final_svr.predict(X_test_scaled)
preds = y_scaler.inverse_transform(preds_scaled)

mse = mean_squared_error(y_test, preds)
rmse = np.sqrt(mse)
print(f"\nTest MSE: {mse:.4f}")
print(f"Test RMSE: {rmse:.4f}")

# Sample prediction
sample = pd.DataFrame([[300, 0.25, 500, 40, 75]], columns=['latency_ms', 'cpu_usage_pct', 'memory_usage_pct', 'memory_allocated', 'cpu_allocated'])
sample_scaled = x_scaler.transform(sample)
predicted_scaled = final_svr.predict(sample_scaled)
predicted = y_scaler.inverse_transform(predicted_scaled)

print(f"\nSample Input: {sample.to_dict(orient='records')[0]}")
print(f"Predicted CPU, Memory Usage: CPU={predicted[0][0]:.3f}, Memory={predicted[0][1]:.2f}")


[I 2025-05-05 08:23:21,134] A new study created in memory with name: no-name-1258cd65-d33d-4a0b-870a-5c23921f93ad
[I 2025-05-05 08:23:23,275] Trial 0 finished with value: 5.538862979324415e-06 and parameters: {'C': 0.349671391422002, 'epsilon': 0.0021487980211985502, 'gamma': 'auto'}. Best is trial 0 with value: 5.538862979324415e-06.
[I 2025-05-05 08:23:24,173] Trial 1 finished with value: 1.2524821332833983e-05 and parameters: {'C': 0.6168715492102821, 'epsilon': 0.005581518409362696, 'gamma': 'auto'}. Best is trial 0 with value: 5.538862979324415e-06.
[I 2025-05-05 08:23:24,246] Trial 2 finished with value: 0.0007188775571164345 and parameters: {'C': 63.90834263133836, 'epsilon': 0.05543265310194885, 'gamma': 'scale'}. Best is trial 0 with value: 5.538862979324415e-06.
[I 2025-05-05 08:23:24,271] Trial 3 finished with value: 0.10155186015794056 and parameters: {'C': 1.2617646449171522, 'epsilon': 0.3997272770721158, 'gamma': 'scale'}. Best is trial 0 with value: 5.538862979324415e-0

Best Parameters: {'C': 41.82060837827947, 'epsilon': 0.0010164043253329798, 'gamma': 'auto'}

Test MSE: 0.0004
Test RMSE: 0.0206

Sample Input: {'latency_ms': 300, 'cpu_usage_pct': 0.25, 'memory_usage_pct': 500, 'memory_allocated': 40, 'cpu_allocated': 75}
Predicted CPU, Memory Usage: CPU=64.748, Memory=29.61
